# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-06-14&until-occurred-date=2021-06-21


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

13565 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-14&until-occurred-date=2021-06-21
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=3ac2192c-6a69-4a9f-a327-1e3e397be8a9&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-14&until-occurred-date=2021-06-21
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=228252a6-459e-4ecc-a124-787190c85893&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-14&until-occurred-date=2021-06-21
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=8e9f3bc7-40b0-4d25-99f9-c85d6ed89fec&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-06-14.csv
failed to load preprint_tweets_2021-06-07.csv
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-31.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.01.11.425914...
REST API query complete  200
REST API query started for 10.1101/2021.06.12.21258835...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/267120...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.06.09.447686...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2020.12.21.423721...
REST API query complete  200
REST API query started for 10.1101/2021.05.22.21257658...
REST API query complete  200
REST API query started for 10.1101/2021.04.15.21252192...
REST API query complete  200
REST API query started for 10.1101/2021.04.20.21255677...
REST API query complete  200
REST API query started for 10.1101/2021.05.30.21257971...
REST A

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.06.12.21258835,1602,medRxiv,Epidemiology,True,Predicted domination of variant Delta of SARS-...,"[{'name': 'Kimihito Ito'}, {'name': 'Chayada P...",<p>Using nucleotide sequences of SARS-CoV-2 st...,2021-06-15
1,10.1101/2021.06.14.448344,160,bioRxiv,Genomics,False,Mapping the Genetic Landscape of DNA Double-st...,"[{'name': 'Jeffrey A. Hussmann'}, {'name': 'Ji...",<p>Cells repair DNA double-strand breaks (DSBs...,2021-06-14
2,10.1101/2021.06.15.448551,151,bioRxiv,Biochemistry,True,Zinc2+ ion inhibits SARS-CoV-2 main protease a...,"[{'name': 'Love Panchariya'}, {'name': 'Wajaha...",<p>Zinc deficiency is linked to poor prognosis...,2021-06-15
3,10.1101/2021.06.10.21258468,118,medRxiv,Epidemiology,True,A tale of two variants: Spread of SARS-CoV-2 v...,"[{'name': 'Christian L. Althaus'}, {'name': 'S...",<p>Several SARS-CoV-2 variants of concern (VOC...,2021-06-15
4,10.1101/2021.06.18.448989,91,bioRxiv,Neuroscience,False,The functional specialization of visual cortex...,"[{'name': 'Shahab Bakhtiari'}, {'name': 'Patri...",<p>The visual system of mammals is comprised o...,2021-06-18
5,10.1101/2021.06.16.21259052,76,medRxiv,Epidemiology,True,Rapid spread of the SARS-CoV-2 δ variant in th...,"[{'name': 'Samuel Alizon'}, {'name': 'Stephani...","<p>Analysing 5,061 variant-specific tests perf...",2021-06-20
6,10.1101/2021.06.16.448638,48,bioRxiv,Neuroscience,False,State transitions in the statistically stable ...,"[{'name': 'Sander Tanni'}, {'name': 'William d...",<p>The hippocampus plays a central role in mam...,2021-06-17
7,10.1101/2021.06.16.448697,45,bioRxiv,Cell Biology,False,Visualizing looping of two endogenous genomic ...,"[{'name': 'Yang Liu'}, {'name': 'Ning Zhao'}, ...","<p>In eukaryotic nuclei, chromatin loops media...",2021-06-16
8,10.1101/2021.06.12.21258345,41,medRxiv,Pharmacology and Therapeutics,True,Coadministration of AYUSH 64 as an adjunct to ...,"[{'name': 'Arvind Chopra'}, {'name': 'Girish T...",<sec><title>Objectives</title><p>To compare th...,2021-06-17
9,10.1101/2021.06.16.448755,34,bioRxiv,Neuroscience,False,Neural event segmentation of continuous experi...,"[{'name': 'Tristan S. Yates'}, {'name': 'Lena ...","<p>Although sensory input is continuous, we pe...",2021-06-16


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

50 preprints (including 17 covering SARS-CoV-2, 40 from bioRxiv and 10 from medRxiv) published in the last 7 days before 2021-06-21 had been tweeted at least 5 times (maximum 1602).